In [9]:

!pip install --upgrade google-cloud



You should consider upgrading via the '/Users/syoung/opt/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip install google-cloud-bigquery

!gcloud --version

     |████████████████████████████████| 247 kB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 23.4 MB/s eta 0:00:01
     |████████████████████████████████| 210 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 293 kB 51.0 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 417 kB 41.0 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 40.9 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 53.4 MB/s eta 0:00:01
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
     |████████████████████████████████| 83 kB 3.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/syoung/opt/anaconda3/bin/python3 -m pip install --upgrade pip' command.
Google Cloud SDK 516.0.0
bq 2.1.14
core 2025.03.24
gcloud-crc32c 1.0.0
gsutil 5.33


In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

# Construct a BigQuery client object.
client = bigquery.Client(project="physionet-data")

# List datasets in the specified project
datasets = list(client.list_datasets())

# Print the dataset names
for dataset in datasets:
    print(dataset.dataset_id)

# TODO(developer): Set table_id to the ID of the destination table.
# table_id = "your-project.your_dataset.your_table_name"
sql = "SELECT * FROM `physionet-data.mimiciii_demo.patients`"
query_job = client.query(sql)

# Wait for the query to finish and fetch the results into a Pandas DataFrame
df = query_job.to_dataframe()

plt.figure(figsize=(8, 6))
df['GENDER'].value_counts().plot(kind='bar', color='skyblue')

plt.title('Gender Distribution in MIMIC-III Patient Dataset')
plt.xlabel('Gender')
plt.ylabel('Count')

plt.show()

/Users/syoung/opt/anaconda3/lib/python3.8/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


eicu_crd_demo
mimiciii_clinical
mimiciii_demo
mimiciii_derived
mimiciii_notes


Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/physionet-data/jobs?prettyPrint=false: Access Denied: Project physionet-data: User does not have bigquery.jobs.create permission in project physionet-data.

Location: None
Job ID: 28f8e18d-0223-4ce8-8357-6b5f6d41d25e


In [7]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project="physionet-data")

# List datasets in the specified project
datasets = list(client.list_datasets())

# Print the dataset names
for dataset in datasets:
    print(dataset.dataset_id)

# TODO(developer): Set table_id to the ID of the destination table.
# table_id = "your-project.your_dataset.your_table_name"

sql = "SELECT * FROM `physionet-data.mimiciii_demo.patients`"
cluster_fields = ["GENDER"]

job_config = bigquery.QueryJobConfig(
    clustering_fields=cluster_fields, destination=table_id
)

# Start the query, passing in the extra configuration.
client.query_and_wait(
    sql, job_config=job_config
)  # Make an API request and wait for job to complete.

table = client.get_table(table_id)  # Make an API request.
if table.clustering_fields == cluster_fields:
    print(
        "The destination table is written using the cluster_fields configuration."
    )

/Users/syoung/opt/anaconda3/lib/python3.8/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


eicu_crd_demo
mimiciii_clinical
mimiciii_demo
mimiciii_derived
mimiciii_notes


NameError: name 'table_id' is not defined